In [1]:
import scipy as sc
import numpy as np
from scipy.stats import norm
import math as m

In [3]:
def ccm_unred(wave, flux, ebv, r_v=""):
    '''Deredden a flux vector using the Cardelli, Clayton, & Mathis, 1989 ApJ,345, 245 prescription.
    Includes the update for the near-UV given by O'Donnell, 1994, ApJ, 422, 158.
    Parameterization is valid from the IR to the far-UV (3.5 microns to 0.1 microns).
    NOTE: Wavelength in Angstroms
    Function converted from IDL, not by me!
    '''
    wave = np.array(wave, float)
    flux = np.array(flux, float)
    
    if wave.size != flux.size: 
        raise TypeError, 'ERROR - wave and flux vectors must be the same size'
    
    if not bool(r_v): 
        r_v = 3.1 

    x = 10000.0/wave
    npts = wave.size
    a = np.zeros(npts, float)
    b = np.zeros(npts, float)
    
    ###############################
    #Infrared
    
    good = np.where( (x > 0.3) & (x < 1.1) )
    a[good] = 0.574 * x[good]**(1.61)
    b[good] = -0.527 * x[good]**(1.61)
    
    ###############################
    # Optical & Near IR

    good = np.where( (x  >= 1.1) & (x < 3.3) )
    y = x[good] - 1.82
    
    c1 = np.array([0.32999,-0.77530,0.01979,0.72085,-0.02427,-0.50447,0.17699,1.0])
    c2 = np.array([-2.09002,5.30260,-0.62251,-5.38434,1.07233,2.28305,1.41338,0.0] )

    a[good] = np.polyval(c1, y)
    b[good] = np.polyval(c2, y)

    ###############################
    # Mid-UV
    
    good = np.where( (x >= 3.3) & (x < 8) )   
    y = x[good]
    F_a = np.zeros(np.size(good), float)
    F_b = np.zeros(np.size(good), float)
    good1 = np.where( y >= 5.9 )    
    
    if np.size(good1) > 0:
        y1 = y[good1] - 5.9
        F_a[ good1] = -0.04473 * y1**2 - 0.009779 * y1**3
        F_b[ good1] =   0.2130 * y1**2  +  0.1207 * y1**3

    a[good] =  1.752 - 0.316*y - (0.104 / ( (y-4.67)**2 + 0.341 )) + F_a
    b[good] = -3.090 + 1.825*y + (1.206 / ( (y-4.62)**2 + 0.263 )) + F_b
    
    ###############################
    # Far-UV
    
    good = np.where( (x >= 8) & (x <= 10) )   
    y = x[good] - 8.0
    c1 = [-0.070 ,0.137,-0.628, -1.073]
    c2 = [0.374 ,-0.420,4.257,13.670]
    a[good] = np.polyval(c1, y)
    b[good] = np.polyval(c2, y)

    # Applying Extinction Correction
    
    a_v = r_v * ebv
    a_lambda = a_v * (a + b/r_v)
    funred = flux * 10.0**(0.4*a_lambda) 

    return(funred,a,b,x)
def confidenceInterval(y,sig):
    '''Calculates the Gaussian sigma confidence interval for a pdf'''
    median=np.percentile(y,50)
    pct15=np.percentile(y,15)
    pct85=np.percentile(y,85)
    list1=np.array([median,median-pct15,pct85-median])
    return list1
def magToFlux(mag,zeroP):
    '''Convert magnitudes to flux units'''
    f=[]
    for item in mag:
        fv=zeroP*10**(-item/2.5)
        f.append(fv)
    return(f)

In [10]:
def MCdeRed(val,lamb,flag,zeroP='',nH='',ebmv=''):
    '''Monte Carlo version of going from magnitude/flux to dereddend flux in Jy.
    Can use:
    (a) E(B-V) value directly, or
    (b) nH value and the relation between optical extinction 
    and hydrogen column density in the Galaxy (Guver & Ozel, 2009, MNRAS, 400, 2050)

    INPUT: 
    val=magnitude or flux (in ergs^-1cm^-2hz^-1); list, [val,err]
    lamb=wavelength in microns
    flag='mag' or'flux' depending on input data values
    zeroP=zero point in Jy (only set if inputting magnitude for val)
    nH=column density in cm^-2; list [val,err] (only set if using nH value method)
    ebmv=E(B-V) value (only set if using E(B-V) value method)
    
    OUTPUT: 
    fEST=dereddend flux list, [flux,lower error, upper error], all in Jy
    '''
    lamB=lamb*10000.0
    samples=np.random.normal(0,1,100)
    samples2=np.random.normal(0,1,100)
    flux=[]
    for ii in range(0,len(samples)):
        if flag=='mag':
            mag_sim = samples[ii]*val[1]+val[0]
            flux_sim=magToFlux([mag_sim],zeroP)
        elif flag=='flux':
            flux_sim = samples[ii]*val[1]+val[0]
        if nH!='':
            nh_sim  = samples2[ii]*nH[1]+nH[0]
            av=nh_sim/(2.21*10**21)
            ebv_sim=av/3.1
        elif ebmv!='':
            ebv_sim  = ebmv[0]#samples2[ii]*ebmv[1]+ebmv[0]
        fluxUn=ccm_unred([lamB],[flux_sim], ebv_sim, r_v=3.1)
        fluxUn1=fluxUn[0]
        flux.append(fluxUn1[0])
    if flag=='mag':
        fEST=confidenceInterval(flux,1)
    elif flag=='flux':
        fEST=confidenceInterval(flux,1)*1e23
    return(fEST)

In [11]:
#examples

#(1) mag to deredended flux using nH
print MCdeRed([10.2,0.1],2.190,'mag',zeroP=640.0,nH=[3*10**20,0.5*10**20])

#(2) flux to deredended flux using nH
print MCdeRed([0.00256*10**-26,0.001e-26],0.310,'flux',nH=[8.1*10**21,0.1*10**21])

#(3) flux to deredended flux using E(B-V)
print MCdeRed([(1.68654e-16/(3e18))*(0.3465*10000)**2,(6.60284e-18/(3e18))*(0.3465*10000)**2],\
          0.310,'flux',ebmv=[1.2,0.2])

#(4) mag to deredended flux using using E(B-V)
print MCdeRed([25.,0.1],0.763,'mag',zeroP=3631.0,ebmv=[1.1,0.1])

[ 0.05568547  0.0055773   0.00489127]
[ 0.00090849  0.00048556  0.00048901]
[ 0.02839986  0.00138225  0.00126581]
[  2.85164846e-06   2.79391120e-07   2.22742912e-07]
